## 1. Settings of Notebook

In [13]:
import time

URL = "http://localhost:3002"


## 2. Loading of materials

In [14]:
material_json_1 = {
    "name": "New Material",
    "basis": {
        "elements": [
            {"id": 0, "value": "Si"},
            {"id": 1, "value": "Si"}
        ],
        "coordinates": [
            {"id": 0, "value": [0, 0, 0]},
            {"id": 1, "value": [0.25, 0.25, 0.25]}
        ],
        "units": "crystal",
        "cell": [
            [3.34892, 0, 1.9335],
            [1.116307, 3.157392, 1.9335],
            [0, 0, 3.867]
        ],
        "constraints": []
    },
    "lattice": {
        "a": 3.867, "b": 3.867, "c": 3.867,
        "alpha": 60, "beta": 90, "gamma": 90,
        "units": {"length": "angstrom", "angle": "degree"},
        "type": "FCC",
        "vectors": {
            "a": [3.34892, 0, 1.9335],
            "b": [1.116307, 3.157392, 1.9335],
            "c": [0, 0, 3.867],
            "alat": 1,
            "units": "angstrom"
        }
    },
    "isNonPeriodic": False
}

material_json_2 = {
    "name": "Another Material",
    "basis": {
        "elements": [
            {"id": 0, "value": "Si"},
            {"id": 1, "value": "O"}
        ],
        "coordinates": [
            {"id": 0, "value": [0, 0, 0]},
            {"id": 1, "value": [0.15, 0.15, 0.15]}
        ],
        "units": "crystal",
        "cell": [
            [5.0, 0, 0],
            [0, 5.0, 0],
            [0, 0, 5.0]
        ],
        "constraints": []
    },
    "lattice": {
        "a": 5.0, "b": 5.0, "c": 5.0,
        "alpha": 90, "beta": 90, "gamma": 60,
        "units": {"length": "angstrom", "angle": "degree"},
        "type": "CUB",
        "vectors": {
            "a": [5.0, 0, 0],
            "b": [0, 5.0, 0],
            "c": [0, 0, 5.0],
            "alat": 1,
            "units": "angstrom"
        }
    },
    "isNonPeriodic": False
}

materials_settings = [{
    "material_json": material_json_1,
    "name": "New Material 1 (Si2).gif"
},
{"material_json": material_json_2,
    "name": "New Material 2 (SiO).gif"
}]

## 3. Definitions

In [21]:
from IPython.display import display, IFrame, Javascript, HTML
import json

import time
import os


def send_message_to_iframe(message):
    """
    Creates and displays an iframe pointing to `url`,
    then sends `message` to the iframe via postMessage.
    """
    js_code = f"""
    (function() {{
        const iframe = document.querySelector('iframe');
        if (iframe && iframe.contentWindow) {{
            try {{
                iframe.contentWindow.postMessage({json.dumps(message)}, '*');
                console.log('Message sent to iframe');
            }} catch (error) {{
                alert('Error sending message to iframe. See console for more information.', error);
            }}
        }} else {{
            console.error('Iframe not found or not ready');
        }}
    }})();
    """
    display(Javascript(js_code))


def set_material_in_iframe(material_json):
    """
    Uses send_message_to_iframe to send a material configuration
    under the "material" key to the iframe at `url`.
    """
    send_message_to_iframe({"material": material_json})


def record_gif(filename, rotation_speed=60, frame_duration=0.05):
    """
    Uses send_message_to_iframe to send a message to the iframe at `url`
    to start recording a GIF of the visualization with the specified parameters.
    """
    message = {
        "action": "handleStartGifRecording",
        "parameters": [filename, rotation_speed, frame_duration]
    }
    send_message_to_iframe(message)

def set_camera( pos=(0, 5, 0), target=(0, 0, 0)):
    func_str = f"(wave) => {{ alert('set_camera'); wave.camera.position.set({pos[0]}, {pos[1]}, {pos[2]}); wave.rebuildScene(); }}"
    message = {
        "action": "doFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)

def set_camera_to_fit_cell():
    message = {
        "action": "handleSetCameraToFitCell",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def toggle_bonds():
    message = {
        "action": "handleToggleBonds",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def handle_reset_viewer():
    message = {
        "action": "handleResetViewer",
        "parameters": [None]
    }
    send_message_to_iframe(message)

def test_do_func():
    func_str = "() => { alert('do func') }"
    message = {
        "action": "doWaveFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)


## 4. Gif Generation

In [ ]:
# Example usage with a material configuration:
iframe = IFrame(src=URL, width="400", height="400")
display(iframe)


In [ ]:
test_do_func()
set_camera_to_fit_cell()

In [ ]:

for material_settings in materials_settings:
    GIF_NAME =  material_settings["name"] or material_settings["material_json"]["name"]
    handle_reset_viewer()
    time.sleep(1)
    set_material_in_iframe(material_settings["material_json"])
    time.sleep(1)
    toggle_bonds()
    time.sleep(1)
    set_camera() # doesn't work
    time.sleep(1)
    set_camera_to_fit_cell() # doesn't work
    time.sleep(1)
    record_gif( GIF_NAME, 60, 0.05)
    # We should wait until the gif is generated and saved before moving to the next material
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
    gif_path = os.path.join(parent_dir, GIF_NAME)
    print(f"Waiting for gif to be generated in {gif_path}")
    while not os.path.exists(gif_path):
        time.sleep(1)
    else:
        print(f"Generated gif for {GIF_NAME}")
    # time.sleep(30)
